<a href="https://colab.research.google.com/github/einsteinxx/UCLA_BIOENG_596_FALL_21/blob/main/rebuild_raw_ultrasound_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import os
import math #for ceil function
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import gc  #debug memory leaks in matplotlib
import csv #read in description files
import random #used to select random slice for patches
import cv2
import re #regexp for BUSI files
from skimage import exposure  #image equalization
import torchvision.transforms as transforms #for imagenet specific stage2 normalization

################################################################################
#ULTRASOUND NEEDS
from bs4 import BeautifulSoup #to read the XML annotation files

import PIL
#from PIL import Image
# Open the image form working directory
#image = Image.open(full_file)

from matplotlib import image
from ast import literal_eval #used to break out bounding boxes from strings
import time


from numpy import clip

################################################################################

eps = 1e-16 #keep div by 0 away


from google.colab.patches import cv2_imshow
#
# Read Data from google drive
#
from google.colab import drive #for loading gdrive data
from google.colab import files

# install dependencies not included by Colab
# use pip3 to ensure compatibility w/ Google Deep Learning Images 
!pip3 install -q pydicom 
!pip3 install -q tqdm 
!pip3 install -q imgaug
!pip3 install -q pickle5
#for beautifulsoup
!pip3 -q install beautifulsoup4

#for lmxl parser
!pip3 -q install lxml

import pydicom #to read dicom files
from pydicom import dcmread
import pickle5 as pickle; #generic storage of image array
import shutil #copy file libraries

################################################################################
# Load data from google drive
################################################################################
drive.mount('/content/gdrive')

#
# Setup of input and output drives
#

#
# Choose training sets to use
#
training_set = 0 #0=UCLA US, 1 = UCLA + BUSI, 2 = BUSI
copy_files = 0 #puts the batch1 and batch2 files in one folder

#storage areas 
data_dir = '/content/gdrive/Shareddrives/BreastUS'
local_dir = '/content/gdrive/My Drive/BreastUS' #for local storage
model_dir = '/content/gdrive/My Drive/BreastUS/MODEL_SAVE'
tensorboard_dir =  '/content/gdrive/My Drive/BreastUS/TENSORBOARD_SUMMARIES'


#
# UCLA specific data folders
#
csv_dir = os.path.join(data_dir,'Annotated data')

annotated_dir = os.path.join(data_dir,'Annotated data')

data_files = os.listdir(annotated_dir)
label_data_dir = os.path.join(annotated_dir,'LabelMe_3.0_format_updated')
label_files = os.listdir(label_data_dir)

#
# Alternative set of Ultrasound data
#
busi_main_dir = '/content/gdrive/My Drive/BreastUS/BUSI_DATA' #downloaded BUSI

#output converted BUSI images go in here:
#norm_dir = '0TO1NORM'  #-2 to 2 norm method is N2TO2_NORM\
norm_dir = 'N2TO2_NORM'
busi_converted_main_dir = os.path.join('/content/gdrive/My Drive/BreastUS/',norm_dir,'BUSI_DATA_CONTRAST_EQUALIZED')
ucla_converted_main_dir = os.path.join('/content/gdrive/My Drive/BreastUS/',norm_dir,'UCLA_DATA_CONTRAST_EQUALIZED')



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Extract Annotations from XML

In [ ]:


def extract_bb(full_file):
    corner = []
 

    #full_file = xml_list[30]
    # Reading the data inside the xml file to a variable under the name  data
    with open(full_file, 'r') as f:
        data = f.read() 


    # Passing the stored data inside the beautifulsoup parser 
    bs_data = BeautifulSoup(data, 'xml') 

    # Finding all instances of tag   
    nrows = bs_data.find_all('nrows') 
    ncols = bs_data.find_all('ncols') 
    boundingbox = bs_data.find_all('bounding_box')
    xx = bs_data.find_all('x')
    yy = bs_data.find_all('y')
    #print(nrows) 

    #for ii in [nrows,ncols]:
    #    print(ii[0])
    #    print(ii[0].get_text())


    xpoints=[]
    ypoints=[]

    for item in xx:
        xpoints.append(int(float(item.text)))
    for item in yy:
        ypoints.append(int(float(item.text)))

    max_points = 4
    if ((len(xpoints)<max_points) or (len(ypoints)<max_points)):
        pass
        #print('Not enough annotations ',len(xpoints), len(ypoints))

    else: #proceed to make new corner positions

        
        for num in range(0,4):
            #order needed for ML model is x,y
            corner.append((xpoints[num],ypoints[num]))
            #corner.append((ypoints[num],xpoints[num]))

        #for item in boundingbox:
        #    pt = item.find('pt').text
        #    print('pt is ',pt)
    return corner
'''
    stop
########### Title #############
required0 = soup.find_all("title")
title = []
for i in required0:
    title.append(i.get_text())

# Using find() to extract attributes of the first instance of the tag 
b_name = bs_data.find('child', {'name':'Acer'}) 
print(b_name) 

# Extracting the data stored in a specific attribute of the `child` tag 
value = b_name.get('qty') 
print(value)
'''

'\n    stop\n########### Title #############\nrequired0 = soup.find_all("title")\ntitle = []\nfor i in required0:\n    title.append(i.get_text())\n\n# Using find() to extract attributes of the first instance of the tag \nb_name = bs_data.find(\'child\', {\'name\':\'Acer\'}) \nprint(b_name) \n\n# Extracting the data stored in a specific attribute of the `child` tag \nvalue = b_name.get(\'qty\') \nprint(value)\n'

# Get CSV Data Function

In [ ]:
def get_csv_data(filename):
    fields = []
    rows = []

    # reading csv file
    with open(filename, 'r') as csvfile:
        # creating a csv reader object
        csvreader = csv.reader(csvfile)
        
        # extracting field names through first row
        fields = next(csvreader)

        # extracting each data row one by one
        for row in csvreader:
            rows.append(row)


        # lines present
        print("found rows: %d"%(csvreader.line_num))
    return fields, rows

#Process Batch2 raw files

In [ ]:
data_files
#annotated_dir for batch2 area
b2_dir = os.path.join(annotated_dir,'unzipped_Batch2')

csv1 = 'annotations_updated_final.csv'
csv_file = os.path.join(csv_dir,csv1)
csv2='case_list_batch2_w_diagnosis.csv' #batch2 csv info
csv2_file = os.path.join(data_dir,csv2)

#
# Get Batch2 CSV information to assign to new rows
#
fields2, rows2 = get_csv_data(csv2_file)
fields, rows = get_csv_data(csv_file)

#patient directories
pdir = os.listdir(b2_dir)

patient_data={}
for pt_list in pdir:
    patient_data[pt_list] ={}
    patient_data[pt_list]['frame_id'] ={} #store id and corner data under this
    print(pt_list)

    

    patient = os.path.join(b2_dir, pt_list)

    #--- add in the remaining folders
    #These batch2 datasets have a default/default path added
    default = os.path.join(patient,'default','default')

    if (os.path.exists(default) == True):
        print('!!!! double default found !!!!')
        pass
    else:
        #some folders may not have default/default, so remove one of them and go
        default = os.path.join(patient,'default')

    files = os.listdir(default)
    print(default)
    xml_list =[]
    for ii in files:
        
        if ('.xml' in ii):
            temp = ii.split('_')
            number = re.findall('\d+',temp[1])

            
            #frame_id = int(number[0])
            
            
            full_xml_file = os.path.join(default,ii)
            corner_data = extract_bb(full_xml_file)
            patient_data[pt_list]['frame_id'][int(number[0])] ={}
            
            patient_data[pt_list]['frame_id'][int(number[0])]['corner_data'] = corner_data
            
            #patient_data[pt_list]['frame_id'] = corner_data
            image_ii = ii.replace('.xml', '.png')
            patient_data[pt_list]['frame_id'][int(number[0])]['image_path'] = os.path.join(default,image_ii)
            #patient_data[pt_list]['image_path'] = os.path.join(default,image_ii)
            print(corner_data)
            #xml_list.append(os.path.join(default,ii))
            



Streaming output truncated to the last 5000 lines.
[]
[]
[]
1_s8684rjf_a_h2vrv14e_2.mp4-2022_01_31_19_18_43-labelme 3.0.zip
/content/gdrive/Shareddrives/BreastUS/Annotated data/unzipped_Batch2/1_s8684rjf_a_h2vrv14e_2.mp4-2022_01_31_19_18_43-labelme 3.0.zip/default
[(431, 238), (599, 238), (599, 390), (431, 390)]
[(431, 238), (605, 238), (605, 390), (431, 390)]
[]
[(431, 238), (599, 238), (599, 390), (431, 390)]
[]
[]
[(455, 242), (631, 242), (631, 407), (455, 407)]
[(455, 239), (631, 239), (631, 407), (455, 407)]
[]
[(418, 238), (586, 238), (586, 390), (418, 390)]
[]
[(460, 241), (610, 241), (610, 409), (460, 409)]
[(455, 230), (631, 230), (631, 407), (455, 407)]
[(420, 231), (611, 231), (611, 379), (420, 379)]
[]
[]
[(431, 238), (599, 238), (599, 390), (431, 390)]
[(422, 237), (590, 237), (590, 390), (422, 390)]
[]
[]
[(463, 255), (639, 255), (639, 420), (463, 420)]
[]
[]
[]
[(431, 238), (599, 238), (599, 390), (431, 390)]
[]
[]
[]
[(420, 231), (611, 231), (611, 379), (420, 379)]
[]
[

#Create New CSV with Batch2 information

In [ ]:

output_csv = os.path.join(local_dir,'b2.csv')
# open the file in the write mode
f = open(output_csv, 'w')

# create the csv writer
writer = csv.writer(f)

#put header row in data
header_data = ['count','MRM', 'ACCESSION', 'VID_ID', 'FRAME_ID','IMAGE_PATH', \
               'BOUNDING_BOX','FINAL_DIAG','BXSITE','FINAL_DIAG1','FIRST50']
writer.writerow(header_data)


rcounter = 0
# for every patient id, loop through all of the frames
for pt in patient_data.keys():

    print(pt, patient_data[pt])
    print(pt)

    #
    # Get coded mrm, coded accession, and video id
    #
    name = pt.split('_')
    print(name)
    mrm = name[0] +'_'+ name[1]
    mrm = mrm.upper()
    acc = name[2] + '_' + name[3]
    acc = acc.upper()
    video_id = pt

    #
    # Get the frame ids (keys) in order
    #
    klist=[]
    for ii in patient_data[pt]['frame_id'].keys():
        klist.append(ii)
        #print(ii)
        #print(patient_data[pt][ii])
    klist.sort()
    #print(klist)

    #for ii in klist:
    #    print(ii,patient_data[pt][ii])

    #
    # Use sorted frame_id list (klist) to print out row data
    #
    for frame_id in klist: #patient_data[pt]['frame_id'].keys():
        #patient_data[pt_list]['frame_id'][int(number[0])] 
        image_path = patient_data[pt]['frame_id'][frame_id]['image_path']
        box = patient_data[pt]['frame_id'][frame_id]['corner_data']



        #
        # Get Info from Batch2 CSV
        #
        for row_vals in rows2:
            if (mrm in row_vals[0]):
                if (acc in row_vals[1]):
                    print('found a matching mrm/acc')
                    first50 = row_vals[4]
                    finaldiag = row_vals[2]
                    bxsite = row_vals[3]

        if (len(first50) == 0):
            #something went wrong with the comparison, kill this process
            print('failed csv compare')
            stop

        print(mrm)
        print(acc)
        print(video_id)
        row_data = [rcounter,mrm, acc, video_id, frame_id,image_path, box, finaldiag, bxsite, 'N/A',first50]
        print(row_data)
        print(patient_data[pt].keys())

        # write a row to the csv file
        writer.writerow(row_data)
        rcounter+=1

    '''
    #
    # Get the frame ids (keys) in order
    #
    klist=[]
    for ii in patient_data[pt].keys():
        klist.append(ii)
        #print(ii)
        #print(patient_data[pt][ii])
    klist.sort()
    print(klist)

    for ii in klist:
        print(ii,patient_data[pt][ii])

    stop
    '''






# close the file
f.close()

Streaming output truncated to the last 5000 lines.
dict_keys(['frame_id'])
found a matching mrm/acc
1_LHDSMIW4
A_0W9J3G5T
1_lhdsmiw4_a_0w9j3g5t_2.mp4-2022_01_30_22_06_37-labelme 3.0.zip
[2701, '1_LHDSMIW4', 'A_0W9J3G5T', '1_lhdsmiw4_a_0w9j3g5t_2.mp4-2022_01_30_22_06_37-labelme 3.0.zip', 42, '/content/gdrive/Shareddrives/BreastUS/Annotated data/unzipped_Batch2/1_lhdsmiw4_a_0w9j3g5t_2.mp4-2022_01_30_22_06_37-labelme 3.0.zip/default/frame_000042.png', [(401, 193), (491, 193), (491, 242), (401, 242)], 'Fibroepithelial lesion, consistent with fibroadenoma', 'Right Brest, 8:00, 6 cm FTN', 'N/A', 'B']
dict_keys(['frame_id'])
found a matching mrm/acc
1_LHDSMIW4
A_0W9J3G5T
1_lhdsmiw4_a_0w9j3g5t_2.mp4-2022_01_30_22_06_37-labelme 3.0.zip
[2702, '1_LHDSMIW4', 'A_0W9J3G5T', '1_lhdsmiw4_a_0w9j3g5t_2.mp4-2022_01_30_22_06_37-labelme 3.0.zip', 43, '/content/gdrive/Shareddrives/BreastUS/Annotated data/unzipped_Batch2/1_lhdsmiw4_a_0w9j3g5t_2.mp4-2022_01_30_22_06_37-labelme 3.0.zip/default/frame_000043.pn

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
[9017, '1_D8G4KJW4', 'A_T56Q311S', '1_d8g4kjw4_a_t56q311s_3.mp4-2022_01_30_21_23_19-labelme 3.0.zip', 50, '/content/gdrive/Shareddrives/BreastUS/Annotated data/unzipped_Batch2/1_d8g4kjw4_a_t56q311s_3.mp4-2022_01_30_21_23_19-labelme 3.0.zip/default/frame_000050.png', [(191, 223), (456, 223), (456, 337), (191, 337)], 'Fibroadenoma with mild usual hyperplasia and focal apocrine metaplasia', 'Right Breast, oval mass, 1:00, 4 cm FTN', 'N/A', 'B']
dict_keys(['frame_id'])
found a matching mrm/acc
1_D8G4KJW4
A_T56Q311S
1_d8g4kjw4_a_t56q311s_3.mp4-2022_01_30_21_23_19-labelme 3.0.zip
[9018, '1_D8G4KJW4', 'A_T56Q311S', '1_d8g4kjw4_a_t56q311s_3.mp4-2022_01_30_21_23_19-labelme 3.0.zip', 51, '/content/gdrive/Shareddrives/BreastUS/Annotated data/unzipped_Batch2/1_d8g4kjw4_a_t56q311s_3.mp4-2022_01_30_21_23_19-labelme 3.0.zip/default/frame_000051.png', [(199, 223), (453, 223), (453, 337), (199, 337)], 'Fibroadenoma with mild usual hyperplasia and focal

In [ ]:
len([])

0

In [ ]:
print(fields)
print(fields2)
print(len(rows))
print(rows[0])
print('---------------------')
for rr in rows2:
    print(rr)

['', 'coded_mrn', 'coded_accession', 'video_id', 'frame_id', 'image_path', 'bounding box', 'Final Diagnosis', 'Bx Site', 'Final Diagnosis.1', 'First 50']
['\ufeffcoded_mrn', 'coded_accession', 'Final Diagnosis', 'Bx Site', 'Coded Diagnosis']
8842
['0', '1_076UP782', 'A_DCC4675O', '1_076up782_a_dcc4675o_0.mp4-2021_09_03_16_33_59-labelme 3.0.zip', '0', 'drive/MyDrive/Annotated data/unzipped_updated/1_076up782_a_dcc4675o_0.mp4-2021_09_03_16_33_59-labelme 3.0.zip/default/frame_000000.PNG', '[]', 'Breast tissue with pseudoangiomatous stromal hyperplasia (PASH)', 'Left Breast, 11:00, 1 cm FTN', 'Breast tissue with pseudoangiomatous stromal hyperplasia (PASH)', 'B']
---------------------
['1_Z97OU004', 'A_89K78944\n', 'Complex fibroadenoma with sclerosing adenosis, apocrine metaplasia and microcysts', 'Right Breast, mass, 12:00, 5cm FTN', 'B']
['1_7S6UGZFF', 'A_32N0Q57D\n', 'Fibroepithelial lesion, most consistent with fibroadenoma ', 'Right Breast, mass, 11:00, 5cm FTN', 'B']
['1_4SA4ZUV9', 

In [ ]:
temp= ii.split('_')
number = re.findall('\d+',temp[1])
number = int(number[0])
print(ii)
print(number)

AttributeError: ignored

In [ ]:
#!ls -lart '/content/gdrive/Shareddrives/BreastUS/Annotated data/unzipped_Batch2/1_pq8nn45n_a_x4g5x404_2.mp4-2022_01_10_00_50_42-labelme 3.0.zip'

!ls -lart '/content/gdrive/Shareddrives/BreastUS/Annotated data/unzipped_Batch2/1_en9w0q3l_a_z4tp3894_2.mp4-2022_01_10_00_50_31-labelme 3.0.zip'

#PUT BATCH1 and BATCH2 RAW DATA IN ONE FOLDER

In [ ]:

if (copy_files ==0):
    print('Skipping file move for Batch1 and Batch2')
else:
    !pip3 install -q distutils-pytest

    import distutils
    from distutils import dir_util

    #distutils allows copying and overwriting files, while copytree does not. 
    #the distutils import for colab is wonky and exists under -pytest. A better, 
    #more stable alternative would be best
    #!pip3 -q install distutils

    #import distutils
    #from distutils import *
    #.dir_util.copy_tree as copy_tree

    from shutil import copytree

    combine_batches = 1
    docopy = 1
    if (combine_batches == 1):


        batch1_dir = os.path.join(annotated_dir, 'unzipped_updated')
        batch2_dir = os.path.join(annotated_dir,'unzipped_Batch2')
        output_dir = os.path.join(annotated_dir,'unzipped_updated_plus_Batch2')

        batch1_list = os.listdir(batch1_dir)
        batch2_list =os.listdir(batch2_dir)
        full_input_list = []
        for folder_name in batch1_list:
            input_dir_name = os.path.join(batch1_dir,folder_name)
            full_input_list.append(input_dir_name)

        for folder_name in batch2_list:
            input_dir_name = os.path.join(batch2_dir,folder_name)  
            full_input_list.append(input_dir_name)

        print(len(full_input_list))


        if (docopy == 1):
            for input_folder in full_input_list:
                #print(input_folder)
                output_subdir = os.path.join(output_dir,os.path.basename(input_folder))
                
                subfolder = os.path.join(output_subdir,'default')
                raw_files = os.listdir(os.path.join(input_folder,'default'))
                if (os.path.isdir(subfolder) == True):
                    # default directory is there, check to see if files in there
                    flist = os.listdir(subfolder)
                    if (len(flist) == len(raw_files)):
                        #this folder copy likely worked, so skip this folder
                        print('folder already completed: ',len(flist),len(raw_files),subfolder)
                        continue
                    else:
                        print('**** missing final files in default/ dir')
                        #shutil.copytree(input_folder,output_subdir)
                        distutils.dir_util.copy_tree(input_folder,output_subdir)
                        flist = os.listdir(subfolder)
                        print('****  copied REPAIRED input_folder to output: ', len(flist),len(raw_files),input_folder)
                else: #default subfolder not created, do a full copy
                    distutils.dir_util.copy_tree(input_folder,output_subdir)
                    #shutil.copytree(input_folder,output_subdir)
                    flist = os.listdir(subfolder)
                    print('copied input_folder to output: ', len(flist),len(raw_files),input_folder)


        print('Finished copying folders')
        #shutil.copytree("sourcedir", "destination")

In [ ]:
print(input_folder)
'''
!pip3 install -q distutils-pytest

import distutils
from distutils import dir_util
#from distutils import *
#from distutils-pytest import *
dir(distutils)
distutils.dir_util.copy_tree
'''

In [ ]:
if (0):    
    print('starting UCLA conversion')
    dbprint =0
    full_pickle_list=[] #store files sent to pickle

    print('--- Loading UCLA Data Files and Categorizing')
    data_dirs = os.listdir(busi_main_dir)
    print('UCLA directories: ', data_dirs)



    annotation_fields, annotation_rows= get_ucla_csv(csv_dir)
    array_rows, mrn, accession, video_id, frame_id, image_path, bounding_box, \
        diagnosis, biopsy_site, diagnosis2, first50 = sort_UCLA_csv(annotation_rows, annotation_fields)


    ############################################################################
    # Get Folder data
    ############################################################################
    full_file_list=[]

    for file_path in image_path:
        #print(file_path)

        filename = os.path.basename(file_path) 
        [_,fpath] =file_path.split('drive/MyDrive/Annotated data/')
        full_file = os.path.join(annotated_dir,fpath)
        full_file_list.append(full_file)
    print('Number of US files: ', len(full_file_list))
    original_file_length = len(full_file_list)

    video_stats(training_set, video_id, bounding_box)

    ############################################################################
    #
    # Default string subsets for replacement for UCLA folders
    # -to replace the CSV given names
    old_string = '/content/gdrive/Shareddrives/BreastUS/Annotated data/unzipped_updated'
    new_string = ucla_converted_main_dir

